### **⬇ Versión Secuencial (Patrón Monolítico) ⬇**

In [ ]:
from PIL import Image
import os
import time

# Directorios
directorio_original = "imagenes_prueba"  # Carpeta donde están las imágenes originales
directorio_grises = "imagenes_gris_secuencial"  # Carpeta donde guardaremos las imágenes procesadas

# Crear la carpeta de salida si no existe
os.makedirs(directorio_grises, exist_ok=True)

# Función para convertir una imagen a escala de grises y guardarla en la nueva carpeta
def convertir_a_gris(ruta_imagen, nombre_original):
    try:
        imagen = Image.open(ruta_imagen)
        imagen_gris = imagen.convert('L')  # 'L' representa escala de grises

        # Crear nueva ruta en la carpeta de imágenes grises
        nombre_archivo, extension = os.path.splitext(nombre_original)
        ruta_gris = os.path.join(directorio_grises, nombre_archivo + "_gris" + extension)

        imagen_gris.save(ruta_gris)
        print(f"Imagen convertida y guardada en: {ruta_gris}")
    except Exception as e:
        print(f"Error al procesar {ruta_imagen}: {e}")

# Función para procesar imágenes de forma secuencial y guardarlas en la nueva carpeta
def procesar_imagenes_secuencial():
    lista_imagenes = [f for f in os.listdir(directorio_original) if os.path.isfile(os.path.join(directorio_original, f))]

    inicio = time.time()

    for nombre_imagen in lista_imagenes:
        ruta_imagen = os.path.join(directorio_original, nombre_imagen)
        convertir_a_gris(ruta_imagen, nombre_imagen)

    fin = time.time()
    print(f"Tiempo total de procesamiento secuencial: {fin - inicio:.2f} segundos")

# Ejecutar el procesamiento secuencial
procesar_imagenes_secuencial()


### **⬇ Versión Paralela con Threads - ThreadPoolExecutor.map() ⬇**

In [ ]:
import concurrent.futures
from PIL import Image
import os
import time

# Directorios
directorio_original = "imagenes_prueba"  # Carpeta con imágenes originales
directorio_grises = "imagenes_gris_paralelo(Thread)"  # Carpeta donde guardaremos las imágenes en escala de grises

# Crear la carpeta de salida si no existe
os.makedirs(directorio_grises, exist_ok=True)

# Obtener la lista de archivos de imagen
archivos = [os.path.join(directorio_original, f) for f in os.listdir(directorio_original) if os.path.isfile(os.path.join(directorio_original, f))]

# Función para convertir una imagen a escala de grises
def convertir_a_gris(ruta_imagen):
    try:
        imagen = Image.open(ruta_imagen)
        imagen_gris = imagen.convert('L')  # 'L' representa escala de grises

        # Crear nueva ruta en la carpeta de imágenes grises
        nombre_archivo, extension = os.path.splitext(os.path.basename(ruta_imagen))
        ruta_gris = os.path.join(directorio_grises, nombre_archivo + "_gris" + extension)

        imagen_gris.save(ruta_gris)
        print(f"Imagen convertida y guardada en: {ruta_gris}")
    except Exception as e:
        print(f"Error al procesar {ruta_imagen}: {e}")

# Procesamiento paralelo con executor.map()
def procesar_imagenes_paralelo_map():
    inicio = time.time()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(executor.map(convertir_a_gris, archivos))  # Aplica la función en paralelo

    fin = time.time()
    print(f"Tiempo total de procesamiento paralelo (map): {fin - inicio:.2f} segundos")

# Ejecutar el procesamiento en paralelo
procesar_imagenes_paralelo_map()


### **⬇ Versión Paralela con Procesos - ProcessPoolExecutor.map() ⬇**

In [ ]:
import concurrent.futures
from PIL import Image
import os
import time

# Directorios
directorio_original = "imagenes_prueba"  # Carpeta con imágenes originales
directorio_grises = "imagenes_gris_paralelo(Process)"  # Carpeta donde guardaremos las imágenes en escala de grises

# Crear la carpeta de salida si no existe
os.makedirs(directorio_grises, exist_ok=True)

# Obtener la lista de archivos de imagen
archivos = [os.path.join(directorio_original, f) for f in os.listdir(directorio_original) if os.path.isfile(os.path.join(directorio_original, f))]

# Función para convertir una imagen a escala de grises
def convertir_a_gris(ruta_imagen):
    try:
        imagen = Image.open(ruta_imagen)
        imagen_gris = imagen.convert('L')  # 'L' representa escala de grises

        # Crear nueva ruta en la carpeta de imágenes grises
        nombre_archivo, extension = os.path.splitext(os.path.basename(ruta_imagen))
        ruta_gris = os.path.join(directorio_grises, nombre_archivo + "_gris" + extension)

        imagen_gris.save(ruta_gris)
        print(f"Imagen convertida y guardada en: {ruta_gris}")
    except Exception as e:
        print(f"Error al procesar {ruta_imagen}: {e}")

# Procesamiento paralelo con executor.map()
def procesar_imagenes_paralelo_map():
    inicio = time.time()

    with concurrent.futures.ProcessPoolExecutor() as executor:
        list(executor.map(convertir_a_gris, archivos))  # Aplica la función en paralelo

    fin = time.time()
    print(f"Tiempo total de procesamiento paralelo (map): {fin - inicio:.2f} segundos")

# Ejecutar el procesamiento en paralelo
procesar_imagenes_paralelo_map()


### **⬇ Versión Paralela a Nivel de Píxeles - Patrón Parallel Map ⬇**

In [ ]:
import concurrent.futures
from PIL import Image
import os
import time

# Directorios
directorio_original = "imagenes_prueba"
directorio_grises = "imagenes_grises(Pixeles)"

# Crear la carpeta de salida si no existe
os.makedirs(directorio_grises, exist_ok=True)

# Función para convertir un píxel a escala de grises
def convertir_pixel_gris(pixel):
    r, g, b = pixel[:3]  # Ignorar el canal alfa si lo hay
    gris = int(0.299 * r + 0.587 * g + 0.114 * b)  # Conversión a escala de grises
    return (gris, gris, gris)  # Retornar el nuevo píxel en formato RGB

# Función para procesar una imagen a nivel de píxeles en paralelo
def procesar_imagen_por_pixeles(ruta_imagen):
    try:
        imagen = Image.open(ruta_imagen)
        imagen = imagen.convert("RGB")  # Convertir a RGB si no lo está

        pixeles = list(imagen.getdata())  # Obtener todos los píxeles en una lista

        # Paralelizar la conversión de píxeles
        with concurrent.futures.ThreadPoolExecutor() as executor:
            pixeles_grises = list(executor.map(convertir_pixel_gris, pixeles))

        # Crear nueva imagen con los píxeles en escala de grises
        imagen_gris = Image.new("RGB", imagen.size)
        imagen_gris.putdata(pixeles_grises)

        # Guardar la imagen procesada
        nombre_archivo, extension = os.path.splitext(os.path.basename(ruta_imagen))
        ruta_gris = os.path.join(directorio_grises, nombre_archivo + "_gris" + extension)
        imagen_gris.save(ruta_gris)

        print(f"Imagen convertida y guardada en: {ruta_gris}")
    except Exception as e:
        print(f"Error al procesar {ruta_imagen}: {e}")

# Función para procesar todas las imágenes
def procesar_imagenes_paralelo_pixeles():
    lista_imagenes = [os.path.join(directorio_original, f) for f in os.listdir(directorio_original) if os.path.isfile(os.path.join(directorio_original, f))]

    inicio = time.time()

    # Paralelizar el procesamiento de imágenes
    with concurrent.futures.ProcessPoolExecutor() as executor:
        list(executor.map(procesar_imagen_por_pixeles, lista_imagenes))

    fin = time.time()
    print(f"Tiempo total de procesamiento paralelo por píxeles: {fin - inicio:.2f} segundos")

# Ejecutar el procesamiento
procesar_imagenes_paralelo_pixeles()
